In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')


In [2]:
df= pd.read_csv("train.csv", index_col= 0)

In [3]:
df.head(2)

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
id,,,,,,,,,,,,,,,,,,,,,
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0


In [4]:
X = df.drop('smoking',axis=1)
y = df['smoking']


In [5]:
lr= LogisticRegression(random_state=24)

In [6]:
kfold = StratifiedKFold(n_splits=5, random_state=24,shuffle=True)
params = {'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],'C':np.linspace(0.001, 10, 5)}
gcv = GridSearchCV(lr, param_grid=params, scoring='roc_auc', cv=kfold, verbose=3)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END .............C=0.001, solver=lbfgs;, score=0.815 total time=   0.9s
[CV 2/5] END .............C=0.001, solver=lbfgs;, score=0.818 total time=   0.9s
[CV 3/5] END .............C=0.001, solver=lbfgs;, score=0.816 total time=   0.9s
[CV 4/5] END .............C=0.001, solver=lbfgs;, score=0.818 total time=   0.9s
[CV 5/5] END .............C=0.001, solver=lbfgs;, score=0.816 total time=   0.8s
[CV 1/5] END .........C=0.001, solver=liblinear;, score=0.818 total time=   8.2s
[CV 2/5] END .........C=0.001, solver=liblinear;, score=0.823 total time=   3.5s
[CV 3/5] END .........C=0.001, solver=liblinear;, score=0.819 total time=   5.6s
[CV 4/5] END .........C=0.001, solver=liblinear;, score=0.822 total time=   5.3s
[CV 5/5] END .........C=0.001, solver=liblinear;, score=0.820 total time=   7.5s
[CV 1/5] END .........C=0.001, solver=newton-cg;, score=0.832 total time=   2.9s
[CV 2/5] END .........C=0.001, solver=newton-cg

In [12]:
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END .............C=0.001, solver=lbfgs;, score=0.815 total time=   1.0s
[CV 2/5] END .............C=0.001, solver=lbfgs;, score=0.818 total time=   1.0s
[CV 3/5] END .............C=0.001, solver=lbfgs;, score=0.816 total time=   0.8s
[CV 4/5] END .............C=0.001, solver=lbfgs;, score=0.818 total time=   1.0s
[CV 5/5] END .............C=0.001, solver=lbfgs;, score=0.816 total time=   0.9s
[CV 1/5] END .........C=0.001, solver=liblinear;, score=0.818 total time=   8.2s
[CV 2/5] END .........C=0.001, solver=liblinear;, score=0.823 total time=   3.6s
[CV 3/5] END .........C=0.001, solver=liblinear;, score=0.819 total time=   5.4s
[CV 4/5] END .........C=0.001, solver=liblinear;, score=0.822 total time=   5.1s
[CV 5/5] END .........C=0.001, solver=liblinear;, score=0.820 total time=   7.8s
[CV 1/5] END .........C=0.001, solver=newton-cg;, score=0.832 total time=   2.8s
[CV 2/5] END .........C=0.001, solver=newton-cg

In [8]:
pd_cv= pd.DataFrame(gcv.cv_results_)
print(pd_cv.shape)

(30, 15)


In [9]:
####### Inferencing with logistic

In [10]:
best_model= gcv.best_estimator_
test = pd.read_csv("test.csv", index_col=0)

In [11]:
y_pred_prob = best_model.predict_proba(test)[:,1]

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- accident
- brand
- clean_title
- engine
- ext_col
- ...
Feature names seen at fit time, yet now missing:
- ALT
- AST
- Cholesterol
- Gtp
- HDL
- ...


In [ ]:
submit= pd.read_csv("sample_submission.csv")
submit['smoking'] = y_pred

submit.to_csv('ss_5_nov_1_lr.csv',index=False)

In [ ]:
#Gaussian

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=24, test_size=0.3, stratify=y)

In [ ]:
ohe= OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
gb = GaussianNB()
pipe= Pipeline([('OHE', ohe), ('GB',gb)])
pipe.fit(X_train, y_train)
y_pred= pipe.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_pred_prob = pipe.predict_proba(X_test)[:,1]   #y=1 probability, interested in 2nd probability
print(roc_auc_score(y_test, y_pred_prob))

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=24,shuffle=True)
params = {'GB__var_smoothing':np.linspace(1e-15, 1, 10)}
gcv = GridSearchCV(pipe, param_grid=params, scoring='roc_auc', cv=kfold)
gcv.fit(X,y)

In [ ]:
print(gcv.best_params_)
print(gcv.best_score_)